In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=23
dept=6
source="PA"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, dept, specialite, source, time, time_type, accessibilite_exp)

23 6 10 PA 30 HC -0.12


In [3]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [4]:
def get_pa(year, specialite):
    sql = f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
and an.dept_id={dept}
group by pa.id, an.id, i.id
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)


In [5]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [6]:
df = get_by_source(year, specialite, source)
df

Quering PA for year 23 and specialite 10

select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat 
from apl.ps_libreacces_2 ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and ps.code_mode_exercice='L'
and ds.annee=23
and pands.adresse_norm_id is not null
and an.dept_id=6
group by pa.id, an.id, i.id



,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,3568,LEDORE,VANINA,6,86364,60300110,7.008846,43.570037
1,4008,TIMAR,GABOR,6,75235,60850108,7.040872,43.588167
2,4673,PIONNEAU,AUDREY,6,272923,61120000,7.003135,43.673439
3,11222,LANSON,FREDERIC,6,2335,61230105,7.192090,43.671030
4,16807,DITTLOT,PIERRE,6,75209,60882601,7.241556,43.689933
...,...,...,...,...,...,...,...,...
717,1529201,BERGONZO,CHRISTIAN,6,3146,60330103,7.192359,43.773162
718,1534240,SAVAGE,FLORENCE,6,272178,60950102,6.876159,43.638990
719,1540045,CORALLINI TEISSIER,MARIE MADELEINE,6,63059,60881703,7.267766,43.706909
720,1540088,UGOLINI,SANDRA,6,62830,60690113,6.939697,43.634656


In [7]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 676
Nb unique PS 722


In [8]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby("id")["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.875346
2    0.116343
3    0.008310
Name: proportion, dtype: float64

In [9]:
df["weight"] = 1 / df["nb_cabinet"]
df["nb"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
0,3568,LEDORE,VANINA,6,86364,60300110,7.008846,43.570037,3568_43.570037_7.008846,1,1.0,3.0
1,4008,TIMAR,GABOR,6,75235,60850108,7.040872,43.588167,4008_43.588167_7.040872,1,1.0,1.0
2,4673,PIONNEAU,AUDREY,6,272923,61120000,7.003135,43.673439,4673_43.673439_7.003135,1,1.0,6.0
3,11222,LANSON,FREDERIC,6,2335,61230105,7.192090,43.671030,11222_43.67103_7.19209,1,1.0,2.0
4,16807,DITTLOT,PIERRE,6,75209,60882601,7.241556,43.689933,16807_43.689933_7.241556,1,1.0,7.0
5,16864,LAPICQUE,PIERRE,6,1590,60480101,7.315379,43.810132,16864_43.810132_7.315379,1,1.0,5.5
6,23645,BURTE,CAROLE,6,2491,60290111,7.021753,43.552328,23645_43.552328_7.021753,1,1.0,15.0
7,24447,CALEMCZUK,GAELLE,6,77494,61490102,7.313438,43.743446,24447_43.743446_7.313438,1,1.0,12.0
8,30483,GARIBBO,RODOLPHE,6,81978,61040105,7.482690,43.764934,30483_43.764934_7.48269,1,1.0,2.0
9,37613,DANDURAN,MARIE,6,2473,61530000,7.201403,44.066566,37613_44.066566_7.201403,1,1.0,2.0


In [10]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
703,1492391,PEYROU,CLOTILDE,6,63070,60040107,7.122697,43.600420,1492391_43.60042_7.122697,1,1.0,1.0
159,326357,BEN HAMMOUDA,ISSAM,6,75227,60040109,7.097858,43.594409,326357_43.594409_7.097858,1,1.0,2.0
303,629059,MARKARIAN,ANISA,6,2182,60040114,7.121019,43.583567,629059_43.583567_7.121019,1,1.0,8.0
291,598012,MAUCHE,BRUNO,6,1626,60040115,7.127911,43.582501,598012_43.582501_7.127911,1,1.0,1.0
277,554320,ROMAND,MONIQUE,6,105558,60040121,7.110934,43.576438,554320_43.576438_7.110934,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
560,1160172,DRAI,THIERRY,6,2193,61590102,7.306829,43.701077,1160172_43.701077_7.306829,1,1.0,3.0
565,1171597,ROUSSEAU,FRANCOISE,6,2635,61610103,7.123625,43.658047,1171597_43.658047_7.123625,1,1.0,1.0
16,52700,NUNES COUCHOUD,FLAVIA,6,271634,61610105,7.141885,43.640635,52700_43.640635_7.141885,1,1.0,5.0
189,377059,DI VINCENZO,DOMINIQUE,6,182425,61620000,7.650077,44.087979,377059_44.087979_7.650077,2,0.5,0.5


In [11]:
def get_iris_matrix(time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
where id / 10000000 = {dept}
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy()
    return m

iris_matrix = get_iris_matrix(time_type)
# iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    


(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
where id / 10000000 = 6
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000=6 or iris_id_to/10000000=6)
and route_min <= 30) order by "iris1", "iris2"



,iris1,iris2,km,time_hc,time_hp,time,iris
0,40420000,60530000,11,19,20,19,60530000
1,40430000,60240000,8,11,12,11,60240000
2,41700000,60870000,18,24,24,24,60870000
3,41700000,60930000,13,17,18,17,60930000
4,41940000,60450000,11,15,16,15,60450000
...,...,...,...,...,...,...,...
5727,61610107,61280000,8,13,16,13,61280000
5728,61610107,61610107,0,0,0,0,61610107
5729,61620000,61620000,0,0,0,0,61620000
5730,61630000,61630000,0,0,0,0,61630000


In [12]:
def get_pop_iris():
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'
and year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'
and year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,60120105,060120105,06012,A,3136.830308,68.356436,55.185368,179.895476,266.832339,226.432893,571.248108,831.040803,391.639115,351.412985,194.786785
1,60500000,060500000,06050,C,526.000000,14.188627,14.119226,40.517897,47.603916,17.493201,71.924387,145.187085,83.001180,65.334467,26.630014
2,61230108,061230108,06123,A,2512.462276,88.787020,75.776145,135.840215,156.862889,129.702563,508.721610,440.454939,303.481339,433.313157,239.522400
3,60720000,060720000,06072,C,142.000000,0.000000,6.761905,4.829932,4.829932,3.863946,21.251701,27.047619,27.047619,31.877551,14.489796
4,61050101,061050101,06105,A,4007.649525,96.881691,145.234138,320.060918,454.509675,180.143151,475.809676,953.212078,561.152910,592.431015,228.214272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60881606,060881606,06088,A,2621.558372,56.982735,81.433187,104.491854,148.417994,461.133207,399.230815,389.919891,334.254784,416.753195,228.940710
496,60881701,060881701,06088,A,1708.308379,24.463401,43.859704,85.003020,126.388729,282.367384,250.163180,355.063504,193.244615,223.033528,124.721315
497,60881804,060881804,06088,A,2893.228431,85.043040,83.274932,124.543590,225.112876,294.722303,673.422185,570.114686,285.987223,350.796724,200.210873
498,60881805,060881805,06088,A,2156.226647,65.721430,54.233925,76.225674,153.780470,257.227060,430.652161,329.042912,253.515770,331.504051,204.323195


In [13]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,40420000,60530000,11,19,20,19,60530000,060530000,06053,C,143.000000,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
1,40430000,60240000,8,11,12,11,60240000,060240000,06024,C,184.000000,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,36.142857,20.809524,55.857143,17.523810
2,41700000,60870000,18,24,24,24,60870000,060870000,06087,C,40.000000,0.000000,0.000000,0.975610,0.975610,3.902439,2.926829,9.756098,4.878049,14.634146,1.951220
3,41700000,60930000,13,17,18,17,60930000,060930000,06093,C,239.000000,3.807375,3.807219,9.513617,21.786587,5.586911,29.050482,49.592803,52.872493,44.500285,18.482229
4,41940000,60450000,11,15,16,15,60450000,060450000,06045,C,76.000000,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,5.421930,17.169444,22.591374,8.132895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5727,61610107,61280000,8,13,16,13,61280000,061280000,06128,C,3183.000000,73.446029,71.266106,150.417571,257.748290,175.135560,356.529570,697.489318,488.228818,716.500761,196.237977
5728,61610107,61610107,0,0,0,0,61610107,061610107,06161,A,2766.654836,82.759075,107.736875,143.551191,220.049146,225.802665,548.248372,564.569591,348.880862,362.105054,162.952005
5729,61620000,61620000,0,0,0,0,61620000,061620000,06162,C,719.000000,12.197578,12.205697,24.164732,35.752722,23.573353,99.958945,145.300496,122.682899,166.533105,76.630473
5730,61630000,61630000,0,0,0,0,61630000,061630000,06163,C,2021.000000,33.629750,31.626001,66.111340,112.543446,104.807846,273.655406,375.828274,325.880261,468.299735,228.617943


In [14]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)

# apl["time"] = apl["time"].replace(0, 0.5)
apl["accessibilite_weight"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,40420000,60530000,11,19,20,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284
1,40430000,60240000,8,11,12,11,60240000,060240000,06024,C,184.0,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,36.142857,20.809524,55.857143,17.523810,0.267135
2,41700000,60870000,18,24,24,24,60870000,060870000,06087,C,40.0,0.000000,0.000000,0.975610,0.975610,3.902439,2.926829,9.756098,4.878049,14.634146,1.951220,0.056135
3,41700000,60930000,13,17,18,17,60930000,060930000,06093,C,239.0,3.807375,3.807219,9.513617,21.786587,5.586911,29.050482,49.592803,52.872493,44.500285,18.482229,0.130029
4,41940000,60450000,11,15,16,15,60450000,060450000,06045,C,76.0,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,5.421930,17.169444,22.591374,8.132895,0.165299
5,41940000,61350000,8,10,11,10,61350000,061350000,06135,C,199.0,3.660030,7.355643,8.475404,15.581092,7.206532,26.174196,36.195032,35.203046,46.237521,12.911504,0.301194
6,60010000,60010000,0,0,0,0,60010000,060010000,06001,C,93.0,4.000000,4.000000,9.000000,1.000000,2.000000,15.000000,20.000000,12.000000,22.000000,4.000000,1.000000
7,60020000,60020000,0,0,0,0,60020000,060020000,06002,C,51.0,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,12.277778,12.277778,14.166667,2.833333,1.000000
8,60020000,60870000,6,8,9,8,60870000,060870000,06087,C,40.0,0.000000,0.000000,0.975610,0.975610,3.902439,2.926829,9.756098,4.878049,14.634146,1.951220,0.382893
9,60030000,60030000,0,0,0,0,60030000,060030000,06003,C,640.0,9.614569,20.984124,45.775888,60.317381,36.232745,93.143833,152.613079,86.279131,119.923459,15.115791,1.000000


In [15]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
5115,61290000,61290000,0,0,0,0,61290000,061290000,06129,C,301.0,1.104023,4.44903,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.76853,24.811635,1.0


In [16]:
cols = [col for col in apl.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [17]:
yy=min(year, 24)
sql = f"""
select o.* from apl.overrepresentation o
where o.year={yy} and o.psp_spe_snds=1
"""
over = pd.read_sql(sql, config.connection_string)
# todo ajouter la colonne psp_spe_snds dans specialite
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,23,1,1.177783,0.544694,0.406151,0.424549,0.723399,0.937987,0.871391,1.063243,1.519927,2.349511


In [18]:
weights = over.values[0,2:]
weights

array([1.17778319, 0.54469355, 0.40615086, 0.42454876, 0.72339938,
       0.93798686, 0.8713911 , 1.0632434 , 1.51992652, 2.34951058])

In [19]:
apl["pop_gp"] = sum(w * apl[c] for w, c in zip(weights, cols))
# if specialite == 5:
#     apl["pop_gp"] /= 2
apl.sort_values(by='iris2')
apl.head(5)   

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,40420000,60530000,11,19,20,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284,167.697233
1,40430000,60240000,8,11,12,11,60240000,060240000,06024,C,184.0,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,36.142857,20.809524,55.857143,17.523810,0.267135,217.456621
2,41700000,60870000,18,24,24,24,60870000,060870000,06087,C,40.0,0.000000,0.000000,0.975610,0.975610,3.902439,2.926829,9.756098,4.878049,14.634146,1.951220,0.056135,46.893956
3,41700000,60930000,13,17,18,17,60930000,060930000,06093,C,239.0,3.807375,3.807219,9.513617,21.786587,5.586911,29.050482,49.592803,52.872493,44.500285,18.482229,0.130029,261.454411
4,41940000,60450000,11,15,16,15,60450000,060450000,06045,C,76.0,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,5.421930,17.169444,22.591374,8.132895,0.165299,95.619673


In [20]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
5115,61290000,61290000,0,0,0,0,61290000,061290000,06129,C,301.0,1.104023,4.44903,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.76853,24.811635,1.0,317.597803


In [21]:
# apl = apl.drop(["iris", "iris_string", "code_commune"], axis=1)
# apl = apl.drop(cols, axis=1)
apl = apl[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,40420000,60530000,11,19,0.102284,167.697233,143.0,C
1,40430000,60240000,8,11,0.267135,217.456621,184.0,C
2,41700000,60870000,18,24,0.056135,46.893956,40.0,C
3,41700000,60930000,13,17,0.130029,261.454411,239.0,C
4,41940000,60450000,11,15,0.165299,95.619673,76.0,C


In [22]:
apl["iris"] = apl["iris1"].copy()
apl.sort_values(by='iris2')
apl = apl.merge(df, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb
0,40420000,60530000,11,19,0.102284,167.697233,143.000000,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40430000,60240000,8,11,0.267135,217.456621,184.000000,C,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41700000,60870000,18,24,0.056135,46.893956,40.000000,C,41700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41700000,60930000,13,17,0.130029,261.454411,239.000000,C,41700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41940000,60450000,11,15,0.165299,95.619673,76.000000,C,41940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5727,61610107,61280000,8,13,0.210136,3433.937399,3183.000000,C,61610107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5728,61610107,61610107,0,0,1.000000,2781.612861,2766.654836,A,61610107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5729,61620000,61620000,0,0,1.000000,847.038425,719.000000,C,61620000,377059.0,DI VINCENZO,DOMINIQUE,6.0,182425.0,7.650077,44.087979,377059_44.087979_7.650077,2.0,0.5,0.5
5730,61630000,61630000,0,0,1.000000,2386.874405,2021.000000,C,61630000,1398844.0,GERSCHTEIN,JEAN LOUIS,6.0,104924.0,7.595441,44.090197,1398844_44.090197_7.595441,1.0,1.0,2.5


In [23]:
apl["nb"] = apl["nb"].fillna(0)
apl = apl.sort_values(by=['iris1', "iris2"])
apl["nb"].unique()

array([ 0.        ,  1.        ,  2.        ,  8.        , 15.        ,
        0.5       ,  4.        ,  9.        ,  7.        ,  6.        ,
        3.5       ,  3.        ,  5.        ,  6.5       ,  1.5       ,
        5.5       ,  0.66666667, 11.        ,  2.5       ,  7.5       ,
        1.83333333,  4.5       , 13.        , 17.5       ,  6.33333333,
       10.        ,  0.33333333,  8.5       , 12.        ])

In [24]:
apl = apl.sort_values(by='iris1')
apl["wpop"] = apl["accessibilite_weight"] * apl["pop_gp"]
apl["swpop"] = apl.groupby("iris1")["wpop"].transform("sum")
apl["R"] = apl["nb"] / (apl["swpop"] / 100000)
apl.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R
0,40420000,60530000,11,19,0.102284,167.697233,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.152778,17.152778,0.0
1,40430000,60240000,8,11,0.267135,217.456621,184.0,C,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58.090340,58.090340,0.0
2,41700000,60870000,18,24,0.056135,46.893956,40.0,C,41700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.632381,36.628961,0.0
3,41700000,60930000,13,17,0.130029,261.454411,239.0,C,41700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,33.996580,36.628961,0.0
4,41940000,60450000,11,15,0.165299,95.619673,76.0,C,41940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.805826,81.382298,0.0


In [25]:
rgp = apl[apl["iris1"] == apl["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
6,60010000,60010000,C,99.509584,0.0,0.000000,99.509584,99.509584,99.509584,93.000000
7,60020000,60020000,C,59.511871,0.0,0.000000,77.467233,59.511871,59.511871,51.000000
9,60030000,60030000,C,623.042768,0.0,0.000000,805.283099,623.042768,623.042768,640.000000
11,60040101,60040101,C,1729.109703,0.0,0.000000,11096.701030,1729.109703,1729.109703,1763.364267
20,60040102,60040102,A,2534.731392,0.0,0.000000,15623.246500,2534.731392,2534.731392,2534.265666
...,...,...,...,...,...,...,...,...,...,...
5700,61610105,61610105,A,3972.608881,5.0,34.131309,14649.306494,3972.608881,3972.608881,3584.453834
5714,61610106,61610106,A,3820.237931,0.0,0.000000,15665.192055,3820.237931,3820.237931,3539.143531
5728,61610107,61610107,A,2781.612861,0.0,0.000000,16534.276185,2781.612861,2781.612861,2766.654836
5729,61620000,61620000,C,847.038425,0.5,59.029199,847.038425,847.038425,847.038425,719.000000


In [26]:
apl = apl.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,40420000,60530000,11,19,0.102284,167.697233,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.152778,17.152778,0.0,60530000,C,167.697233,0.0,0.0,167.697233,167.697233,167.697233,143.0
1,60530000,60530000,0,0,1.000000,167.697233,143.0,C,60530000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,167.697233,167.697233,0.0,60530000,C,167.697233,0.0,0.0,167.697233,167.697233,167.697233,143.0
2,40430000,60240000,8,11,0.267135,217.456621,184.0,C,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58.090340,58.090340,0.0,60240000,C,217.456621,0.0,0.0,217.456621,217.456621,217.456621,184.0
3,60240000,60240000,0,0,1.000000,217.456621,184.0,C,60240000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,217.456621,217.456621,0.0,60240000,C,217.456621,0.0,0.0,217.456621,217.456621,217.456621,184.0
4,41700000,60870000,18,24,0.056135,46.893956,40.0,C,41700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.632381,36.628961,0.0,60870000,C,46.893956,0.0,0.0,69.680628,46.893956,46.893956,40.0


In [27]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,40420000,60530000,11,19,0.102284,167.697233,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.152778,17.152778,0.0,60530000,C,167.697233,0.0,0.0,167.697233,167.697233,167.697233,143.0,0.0,0.0
2,40430000,60240000,8,11,0.267135,217.456621,184.0,C,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,58.090340,58.090340,0.0,60240000,C,217.456621,0.0,0.0,217.456621,217.456621,217.456621,184.0,0.0,0.0
4,41700000,60870000,18,24,0.056135,46.893956,40.0,C,41700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.632381,36.628961,0.0,60870000,C,46.893956,0.0,0.0,69.680628,46.893956,46.893956,40.0,0.0,0.0
7,41700000,60930000,13,17,0.130029,261.454411,239.0,C,41700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,33.996580,36.628961,0.0,60930000,C,261.454411,0.0,0.0,272.550894,261.454411,261.454411,239.0,0.0,0.0
10,41940000,60450000,11,15,0.165299,95.619673,76.0,C,41940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.805826,81.382298,0.0,60450000,C,95.619673,0.0,0.0,115.014613,95.619673,95.619673,76.0,0.0,0.0


In [28]:
apl = apl[apl["iris1"] == apl["iris2"]]
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
16,60010000,60010000,0,0,1.0,99.509584,93.000000,C,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.509584,99.509584,0.000000,60010000,C,99.509584,0.0,0.000000,99.509584,99.509584,99.509584,93.000000,0.000000,0.000000
17,60020000,60020000,0,0,1.0,59.511871,51.000000,C,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,59.511871,77.467233,0.000000,60020000,C,59.511871,0.0,0.000000,77.467233,59.511871,59.511871,51.000000,0.000000,0.000000
19,60030000,60030000,0,0,1.0,623.042768,640.000000,C,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,623.042768,805.283099,0.000000,60030000,C,623.042768,0.0,0.000000,805.283099,623.042768,623.042768,640.000000,0.000000,0.000000
124,60040101,60040101,0,0,1.0,1729.109703,1763.364267,C,60040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1729.109703,11096.701030,0.000000,60040101,C,1729.109703,0.0,0.000000,11096.701030,1729.109703,1729.109703,1763.364267,0.000000,9.248527
248,60040102,60040102,0,0,1.0,2534.731392,2534.265666,A,60040102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2534.731392,15623.246500,0.000000,60040102,A,2534.731392,0.0,0.000000,15623.246500,2534.731392,2534.731392,2534.265666,0.000000,118.212952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,61610105,61610105,0,0,1.0,3972.608881,3584.453834,A,61610105,52700.0,NUNES COUCHOUD,FLAVIA,6.0,271634.0,7.141885,43.640635,52700_43.640635_7.141885,1.0,1.0,5.0,3972.608881,14649.306494,34.131309,61610105,A,3972.608881,5.0,34.131309,14649.306494,3972.608881,3972.608881,3584.453834,34.131309,81.417917
39,61610106,61610106,0,0,1.0,3820.237931,3539.143531,A,61610106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3820.237931,15665.192055,0.000000,61610106,A,3820.237931,0.0,0.000000,15665.192055,3820.237931,3820.237931,3539.143531,0.000000,52.755640
436,61610107,61610107,0,0,1.0,2781.612861,2766.654836,A,61610107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2781.612861,16534.276185,0.000000,61610107,A,2781.612861,0.0,0.000000,16534.276185,2781.612861,2781.612861,2766.654836,0.000000,20.627785
5723,61620000,61620000,0,0,1.0,847.038425,719.000000,C,61620000,377059.0,DI VINCENZO,DOMINIQUE,6.0,182425.0,7.650077,44.087979,377059_44.087979_7.650077,2.0,0.5,0.5,847.038425,847.038425,59.029199,61620000,C,847.038425,0.5,59.029199,847.038425,847.038425,847.038425,719.000000,59.029199,59.029199


In [29]:
print(year, specialite, source)
apl["apl"].describe()
#for 2021
#spe10 psmean=95 vs 98, psmedian=101 vs 110, after damir changes psmean=95 psmedian=99
#spe10 pa104-109
#spe3 ps7.2-7.8 pa7.8-8.4
#spe7 ps6.4-6.6 pa 7.7-7.7
#spe10 newpa 98-104
#spe3 newpa 7.4-7.8
#spe7 newpa 6.2-6.4

23 10 PA


count    500.000000
mean      51.231127
std       55.369880
min        0.000000
25%       10.548078
50%       33.561768
75%       74.998311
max      314.863639
Name: apl, dtype: float64

In [30]:
apl[apl["apl"]>400]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl


In [31]:
apl = apl[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl = apl.rename(columns={'iris1': 'iris'})
apl.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
16,60010000,C,0.0,0.000000,0.0,1.0,99.509584,99.509584,0.0,99.509584,93.000000
17,60020000,C,0.0,0.000000,0.0,1.0,59.511871,77.467233,0.0,59.511871,51.000000
19,60030000,C,0.0,0.000000,0.0,1.0,623.042768,805.283099,0.0,623.042768,640.000000
124,60040101,C,0.0,9.248527,0.0,1.0,1729.109703,11096.701030,0.0,1729.109703,1763.364267
248,60040102,A,0.0,118.212952,0.0,1.0,2534.731392,15623.246500,0.0,2534.731392,2534.265666


In [32]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2023 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,60120105,060120105,Centre,6,06012,Beausoleil,2023,10
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2023,10
2,60720000,060720000,Ilonse,6,06072,Ilonse,2023,10
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2023,10
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2023,10
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2023,10
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2023,10
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2023,10
498,60883602,060883602,Costière,6,06088,Nice,2023,10


In [33]:
apl = apl.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl = apl[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,2023,10,60010000,060010000,C,0.0,0.000000,0.000000,1.0,0.000000,99.509584,99.509584,99.509584,93.000000,Aiglun,6,06001,Aiglun
1,2023,10,60020000,060020000,C,0.0,0.000000,0.000000,1.0,0.000000,59.511871,77.467233,59.511871,51.000000,Amirat,6,06002,Amirat
2,2023,10,60030000,060030000,C,0.0,0.000000,0.000000,1.0,0.000000,623.042768,805.283099,623.042768,640.000000,Andon,6,06003,Andon
3,2023,10,60040101,060040101,C,0.0,9.248527,0.000000,1.0,0.000000,1729.109703,11096.701030,1729.109703,1763.364267,Les Trois Moulins,6,06004,Antibes
4,2023,10,60040102,060040102,A,0.0,118.212952,0.000000,1.0,0.000000,2534.731392,15623.246500,2534.731392,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023,10,61610105,061610105,A,5.0,81.417917,34.131309,1.0,34.131309,3972.608881,14649.306494,3972.608881,3584.453834,Marina,6,06161,Villeneuve-Loubet
496,2023,10,61610106,061610106,A,0.0,52.755640,0.000000,1.0,0.000000,3820.237931,15665.192055,3820.237931,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
497,2023,10,61610107,061610107,A,0.0,20.627785,0.000000,1.0,0.000000,2781.612861,16534.276185,2781.612861,2766.654836,Plan,6,06161,Villeneuve-Loubet
498,2023,10,61620000,061620000,C,0.5,59.029199,59.029199,1.0,59.029199,847.038425,847.038425,847.038425,719.000000,La Brigue,6,06162,La Brigue


In [34]:
# Anomalie = 1 seul medecin avec 1 seul cabinet pour 300 habitants contre 1 seul medecin et 0.33 cabinets
# La commune de sauveur en tinee a un APL > 400 pour les généralistes car l'iris a seulement 300 habitants jeunes et ils sont passés de 1 médecin avec 3 cabinets à 1 médecin a un seul cabinet en 2022
apl[apl["apl"] > 400]


,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label


In [35]:
apl.to_csv("apl06.csv", index=False)